Cannot directly scrape

In [ ]:
from urllib.request import urlopen

url = 'https://www.federalregister.gov/documents/2020/10/14/2020-22666/land-acquisitions-chickasaw-nation'
page = urlopen(url)
print(page)

html_bytes = page.read()
html = html_bytes.decode("utf-8")
print(html)

html_bytes = page.read()
html = html_bytes.decode("utf-8")
print(html)

Need to use API to access raw text URL: <https://www.federalregister.gov/developers/documentation/api/v1>

In [ ]:
import requests

# Define the API endpoint
url = "https://www.federalregister.gov/api/v1/documents.json"

# Define query parameters
params = {
    "conditions[publication_date][gte]": "2024-01-01",  # Start date
    "conditions[publication_date][lte]": "2024-12-31",  # End date
    "conditions[agencies][]": "indian-affairs-bureau",  # Filter by agency
    "conditions[type][]": "NOTICE",
    "conditions[term]": "Land Acquisition",  # Search term
    "fields[]": "raw_text_url",
    "order": "relevance",  # Sort order
    "per_page": 2  # Number of results per page
}

# Make the initial API request
response = requests.get(url, params=params)

# Check for errors
if response.status_code == 200:
    data = response.json()
    raw_text_urls = [doc["raw_text_url"] for doc in data.get("results", [])]
    print(f"Found {len(raw_text_urls)} documents.")
    
    # Process each raw text URL
    for raw_text_url in raw_text_urls:
        print(f"Fetching content from: {raw_text_url}")
        raw_response = requests.get(raw_text_url)
        
        if raw_response.status_code == 200:
            # Extract and print the raw text
            raw_text = raw_response.text
            print(f"Content Preview:\n{raw_text[:2000]}...\n")  # Print the first 1000 characters for preview
            print("-" * 80)
        else:
            print(f"Error fetching raw text: {raw_response.status_code}")
else:
    print(f"Error: {response.status_code} - {response.text}")


In [ ]:
raw_response.text

In [7]:
import geopandas as gpd
from shapely.geometry import Polygon
import re
import math

# Define a function to convert bearing and distance into coordinates
def calculate_new_point(lat, lon, bearing, distance):
    # Convert bearing to radians
    bearing = math.radians(bearing)
    # Radius of the Earth in meters
    R = 6371000  
    # Convert distance to radians
    distance_rad = distance / R
    
    # New latitude in radians
    new_lat = math.asin(math.sin(math.radians(lat)) * math.cos(distance_rad) +
                        math.cos(math.radians(lat)) * math.sin(distance_rad) * math.cos(bearing))
    
    # New longitude in radians
    new_lon = math.radians(lon) + math.atan2(math.sin(bearing) * math.sin(distance_rad) * math.cos(math.radians(lat)),
                                             math.cos(distance_rad) - math.sin(math.radians(lat)) * math.sin(new_lat))
    
    # Convert radians back to degrees
    new_lat = math.degrees(new_lat)
    new_lon = math.degrees(new_lon)
    
    return new_lat, new_lon

# Example function to parse and convert a legal description to a polygon
def parse_legal_description(description):
    # Extract the relevant points and bearings
    points = []
    # Placeholder start point - replace with actual coordinates if known
    start_lat, start_lon = 32.2506, -110.9747  # Example coordinates for Tucson, AZ
    
    current_lat, current_lon = start_lat, start_lon
    for line in description.split('\n'):
        match = re.search(r'([NS]\s*\d+°\d+′\d+″)\s*([EW]\s*\d+°\d+′\d+″)\s*([\d.]+)\s*feet', line)
        if match:
            direction_ns, direction_ew, distance = match.groups()
            distance = float(distance) * 0.3048  # Convert feet to meters
            # Convert bearing to decimal degrees
            bearing = ...  # Conversion logic for bearing
            current_lat, current_lon = calculate_new_point(current_lat, current_lon, bearing, distance)
            points.append((current_lon, current_lat))
    
    # Create a polygon from points
    polygon = Polygon(points)
    return polygon



In [ ]:
# Example legal description
description = """
COMMENCING at the North quarter corner of said Section 2...
THENCE North 89°05′59″ West, 90.07 feet...
THENCE South 00°54′01″ West, 60.00 feet...
THENCE South 00°13′42″ East, 138.47 feet...
THENCE North 89°42′33″ East, 92.00 feet...
THENCE North 00°27′04″ West, 83.83 feet...
"""

description = raw_response.text

# Parse the legal description
 = parse_legal_description(description)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs="EPSG:4326")
# gdf.explore()

# # Save to a file
# gdf.to_file("parcel.shp")  # or .gpkg for GeoPackage

In [ ]:
import logging
import geopandas as gpd
from shapely.geometry import Polygon
import re
import math

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to convert bearing and distance into new coordinates
def calculate_new_point(lat, lon, bearing, distance):
    logging.debug(f"Calculating new point from lat={lat}, lon={lon}, bearing={bearing}, distance={distance}")
    # Convert bearing to radians
    bearing = math.radians(bearing)
    # Radius of the Earth in meters
    R = 6371000  
    # Convert distance to radians
    distance_rad = distance / R
    
    # New latitude in radians
    new_lat = math.asin(math.sin(math.radians(lat)) * math.cos(distance_rad) +
                        math.cos(math.radians(lat)) * math.sin(distance_rad) * math.cos(bearing))
    
    # New longitude in radians
    new_lon = math.radians(lon) + math.atan2(math.sin(bearing) * math.sin(distance_rad) * math.cos(math.radians(lat)),
                                             math.cos(distance_rad) - math.sin(math.radians(lat)) * math.sin(new_lat))
    
    # Convert radians back to degrees
    new_lat = math.degrees(new_lat)
    new_lon = math.degrees(new_lon)
    
    logging.debug(f"New point calculated: lat={new_lat}, lon={new_lon}")
    return new_lat, new_lon

# Function to parse and convert a legal description to a polygon
def parse_legal_description(description):
    logging.debug("Starting to parse the legal description")
    points = []
    start_lat, start_lon = 32.2506, -110.9747  # Example starting coordinates
    current_lat, current_lon = start_lat, start_lon
    points.append((current_lon, current_lat))  # Add starting point
    
    # Filter out non-relevant lines (e.g., HTML tags, metadata, etc.)
    valid_lines = [line for line in description.split('\n') if line.strip() and not line.startswith(('<', '[', 'BILLING'))]
    
    logging.debug(f"Filtered valid lines: {valid_lines}")
    
    # Process each line of the legal description
    for line in valid_lines:
        logging.debug(f"Processing line: {line.strip()}")
        # Adjust the regex to match a broader set of distance and bearing formats
        match = re.search(r'([NS]{1}\s*\d{1,2}[°]?\s*\d{1,2}[′]?\s*\d{1,2}[″]?)\s*([EW]{1}\s*\d{1,2}[°]?\s*\d{1,2}[′]?\s*\d{1,2}[″]?)\s*([\d.]+)\s*(feet|meters)?', line)
        if match:
            direction_ns, direction_ew, distance, _ = match.groups()
            distance = float(distance) * 0.3048 if 'feet' in line else float(distance)  # Convert feet to meters if applicable
            
            # Placeholder for bearing conversion logic
            bearing = ...  # Logic to convert direction_ns, direction_ew to a numeric bearing
            
            logging.debug(f"Direction NS: {direction_ns}, Direction EW: {direction_ew}, Distance: {distance} meters")
            
            current_lat, current_lon = calculate_new_point(current_lat, current_lon, bearing, distance)
            points.append((current_lon, current_lat))
        else:
            logging.warning(f"No match found for line: {line.strip()}")

    # Check if enough points were extracted
    if len(points) < 4:
        logging.error("Not enough points to form a polygon. Check the legal description format.")
        return Polygon([])  # Return an empty polygon

    # Create a polygon from points
    polygon = Polygon(points)
    logging.debug(f"Polygon created with points: {points}")
    return polygon

# Example legal description
description = """
COMMENCING at the North quarter corner of said Section 2...
THENCE North 89°05′59″ West, 90.07 feet...
THENCE South 00°54′01″ West, 60.00 feet...
THENCE South 00°13′42″ East, 138.47 feet...
THENCE North 89°42′33″ East, 92.00 feet...
THENCE North 00°27′04″ West, 83.83 feet...
"""

description = raw_response.text

# Parse the legal description
polygon = parse_legal_description(description)
print(polygon)

# Create a GeoDataFrame
# gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs="EPSG:4326")

# Save to a file
# gdf.to_file("parcel.shp")


Use PLSS

In [ ]:
import geopandas as gpd
import logging

# Set the logging level to WARNING or ERROR to reduce debug verbosity
logging.basicConfig(level=logging.WARNING)  # You can use logging.ERROR for even fewer messages

# Load the PLSS shapefile (replace with the actual file path)
shapefile_path = '/Users/mbraaksma/Downloads/BLM_NATL_PLSS.gdb/ilmocplss.gdb'
gdf = gpd.read_file(shapefile_path, 
                    layer='PLSSIntersected', 
                    where="STATEABBR = 'MN' AND TWNSHPNO = '108' AND TWNSHPDIR = 'N' AND RANGENO = '015' AND RANGEDIR = 'W' AND FRSTDIVNO = '01'", 
                    engine='pyogrio')
#                     where="STATEABBR = 'MN' AND TWNSHPNO = '108' AND TWNSHPDIR = 'N' AND RANGENO = '15' AND RANGEDIR = 'W' AND FRSTDIVNO = '1'", 


# Inspect the columns to find the relevant ones for Township, Range, and Section
print(gdf.columns)

# # Filter for the specific Township, Range, and Section
# # Assuming columns are 'township', 'range', 'section'. Adjust column names based on your shapefile.
# section_1 = gdf[(gdf['township'] == 108) & (gdf['range'] == 15) & (gdf['section'] == 1)]

# # Check the filtered data
# print(section_1)

# # Extract the northwest corner (assuming the section boundary is a polygon)
# # This will return the coordinates of the northwest corner based on the geometry of the section polygon
# if not section_1.empty:
#     northwest_corner = section_1.geometry.unary_union.bounds[0], section_1.geometry.unary_union.bounds[3]
#     print(f"Northwest corner coordinates: {northwest_corner}")
# else:
#     print("Section 1 not found.")


In [ ]:
gdf.drop(columns=['SOURCEDATE', 'REVISEDDATE'], inplace=True)
gdf.explore('FRSTDIVNO')

In [ ]:
import geopandas as gpd
import logging

# Set the logging level to WARNING or ERROR to reduce debug verbosity
logging.basicConfig(level=logging.WARNING)  # You can use logging.ERROR for even fewer messages

# Load the PLSS shapefile (replace with the actual file path)
shapefile_path = '/Users/mbraaksma/Downloads/BLM_NATL_PLSS.gdb/ilmocplss.gdb'
div_gdf = gpd.read_file(shapefile_path, 
                    layer='PLSSIntersected', 
                    where="STATEABBR = 'MN' AND TWNSHPNO = '108' AND TWNSHPDIR = 'N' AND RANGENO = '015' AND RANGEDIR = 'W' AND FRSTDIVNO = '01'", 
                    engine='pyogrio')

In [ ]:
div_gdf.drop(columns=['SOURCEDATE', 'REVISEDDATE']).explore()


In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import folium

def calculate_new_point(start_point, bearing, distance):
    # Convert bearing to radians
    bearing_rad = np.deg2rad(bearing)
    
    # Distance to radians conversion (earth radius approx 6371000 meters)
    distance_rad = distance / 6371000.0

    lat1 = np.deg2rad(start_point.y)
    lon1 = np.deg2rad(start_point.x)

    lat2 = np.arcsin(np.sin(lat1) * np.cos(distance_rad) +
                     np.cos(lat1) * np.sin(distance_rad) * np.cos(bearing_rad))

    lon2 = lon1 + np.arctan2(np.sin(bearing_rad) * np.sin(distance_rad) * np.cos(lat1),
                             np.cos(distance_rad) - np.sin(lat1) * np.sin(lat2))

    return Point(np.rad2deg(lon2), np.rad2deg(lat2))

def create_curve(start_point, radius, central_angle, direction='concave southwest'):
    # Placeholder function for curve handling, to be developed as needed
    # This will require more advanced geometric calculations or interpolation
    # Return a list of points along the curve
    return [start_point]  # Dummy placeholder

# Load the PLSS shapefile
# shapefile_path = '/Users/mbraaksma/Downloads/BLM_NATL_PLSS.gdb/ilmocplss.gdb'
# div_gdf = gpd.read_file(shapefile_path, 
#                         layer='PLSSIntersected', 
#                         where="STATEABBR = 'MN' AND TWNSHPNO = '108' AND TWNSHPDIR = 'N' AND RANGENO = '015' AND RANGEDIR = 'W' AND FRSTDIVNO = '01'", 
#                         engine='pyogrio')

# Get the northwest corner of the section
section_polygon = div_gdf.geometry.unary_union
northwest_corner = Point(section_polygon.bounds[0], section_polygon.bounds[3])

# Trace the described polygon
points = [northwest_corner]
# South 00 degrees 54 minutes 41 seconds East, 778.98 feet
points.append(calculate_new_point(points[-1], 180 + 0.9114, 778.98 * 0.3048))
# South 44 degrees 55 minutes 49 seconds East, 764.84 feet
points.append(calculate_new_point(points[-1], 180 + 44.9303, 764.84 * 0.3048))
# Start of new line, another point at South 44 degrees 55 minutes 49 seconds East, 5121.99 feet
points.append(calculate_new_point(points[-1], 180 + 44.9303, 5121.99 * 0.3048))
# Curve segment with radius 1083.65 feet and central angle 20 degrees 34 minutes 11 seconds
curve_points = create_curve(points[-1], 1083.65 * 0.3048, 20 + 34/60 + 11/3600)
points.extend(curve_points)

# Close the polygon if needed
# points.append(points[0])

# Create a polygon from points
polygon = Polygon(points)

# Visualize or save the new polygon
gdf_polygon = gpd.GeoDataFrame(geometry=[polygon], crs=div_gdf.crs)
# gdf_polygon.to_file("new_polygon.geojson", driver="GeoJSON")

# Plot
m = div_gdf.drop(columns=['SOURCEDATE', 'REVISEDDATE']).explore(name="Section")
m = gdf_polygon.explore(m=m, color="red", name="Parcel")
# this is completely optional
folium.LayerControl().add_to(m)

m
